#1

In [1]:
from requests import get
import re
import string

In [2]:
urls = ['http://lib.ru/PROZA/LIMONOV/r_eastside.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/mussolini.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/r_editor.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/r_excesses.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/r_holiday.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/r_izya.txt_Ascii.txt',
        'https://lib.ru/PROZA/LIMONOV/r_leutenant.txt',
        'http://lib.ru/PROZA/LIMONOV/r_ship.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/r_those.txt_Ascii.txt',
        'http://lib.ru/PROZA/LIMONOV/r_twin.txt_Ascii.txt']

In [3]:
texts = [get(url).text.replace('\x14', ' ').replace('\x15', ' ') for url in urls]

In [4]:
!pip install pymorphy2 --q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.2 MB/s eta 0:00:00


In [5]:
!python -m spacy download ru_core_news_sm --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
!pip install jsonlines

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import spacy
from spacy.lang.ru.examples import sentences
import jsonlines

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
nlp = spacy.load("ru_core_news_sm")

In [9]:
stops = stopwords.words("russian")

tfidf = TfidfVectorizer(
    analyzer="word", # анализировать по словам или по символам (char)
    stop_words=stops # передаём список стоп-слов для русского из NLTK
)

In [10]:
morph = MorphAnalyzer()

In [11]:
texts_preprocessed = ''
for a_text in texts:
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
    texts_preprocessed += a_lemmatized + ' '

In [12]:
doc = nlp(texts_preprocessed)

In [13]:
sw = stopwords.words('russian')

In [14]:
sents_list = []
for sent in doc.sents:
    sentance = sent.text.translate(str.maketrans('', '', string.punctuation))
    sents_list.append(' '.join([w.lower() for w in sentance.split(' ') if w.lower() not in sw]))

In [15]:
with jsonlines.open('texts.jsonl', mode='w') as writer:
    writer.write_all(sents_list)

#2

##2.1

In [18]:
import numpy as np

In [48]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return [(w, tfidf_vector.indices[sorted_nzs][i]) for i, w in enumerate(feature_names[tfidf_vector.indices[sorted_nzs]])]


In [62]:
tfidf_vec = tfidf.fit_transform(texts)
feature_names = np.array(tfidf.get_feature_names_out())
n = 110

top_words = get_top_tf_idf_words(tfidf_vec, feature_names, n)
top_words = dict(sorted(top_words, key=lambda x: x[1], reverse=True))
len(top_words)

100

In [63]:
top_words

{'ян': 13598,
 'эф': 13532,
 'это': 13530,
 'энтони': 13494,
 'эмбасси': 13477,
 'элен': 13475,
 'эдварда': 13430,
 'эдвард': 13429,
 'эвелин': 13426,
 'чаек': 13073,
 'хозяина': 12906,
 'флагом': 12774,
 'флаг': 12770,
 'улицы': 12446,
 'убрать': 12260,
 'товарищи': 12011,
 'тип': 11979,
 'такси': 11801,
 'сэра': 11763,
 'странице': 11539,
 'стив': 11449,
 'стеси': 11440,
 'служанки': 10860,
 'сказала': 10717,
 'сказал': 10716,
 'си': 10627,
 'севу': 10531,
 'севе': 10525,
 'сева': 10524,
 'русский': 10275,
 'ричард': 10140,
 'редактора': 10040,
 'редактор': 10039,
 'пэрэс': 9647,
 'посмотреть': 8626,
 'полях': 8401,
 'полезные': 8275,
 'палубу': 7376,
 'очень': 7330,
 'отец': 7149,
 'отеле': 7142,
 'океана': 6838,
 'общества': 6671,
 'набережной': 5701,
 'мэн': 5682,
 'муссолини': 5660,
 'мусор': 5649,
 'мишка': 5443,
 'матрос': 5274,
 'маленький': 5190,
 'лос': 5062,
 'лицо': 5026,
 'лейтенант': 4921,
 'леди': 4903,
 'лампы': 4873,
 'кэн': 4851,
 'кэмпбэлл': 4849,
 'кудлатый': 4787,

##2.2

In [65]:
texts_vecs = [[int(w in text) for w in top_words.keys()] for text in texts]

In [69]:
for t in texts_vecs:
    print(t)

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 

#3

##3.1

In [70]:
import gensim
import logging
import urllib.request

import warnings
warnings.filterwarnings('ignore')

In [72]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [73]:
f = 'texts.jsonl'
data = gensim.models.word2vec.LineSentence(f)

In [74]:
%time model_texts = gensim.models.Word2Vec(data, vector_size=300, window=5, min_count=5, epochs=50)

CPU times: user 4.4 s, sys: 94.1 ms, total: 4.49 s
Wall time: 4.01 s


##3.2

In [75]:
print(len(model_texts.wv.key_to_index))

1291


маловато

##3.3

In [76]:
love_sim = model_texts.wv.most_similar("любовь", topn=10)
love_sim

[('"думать', 0.8978483080863953),
 ('понимать', 0.8882246613502502),
 ('помнить', 0.8824179768562317),
 ('настаивать', 0.8778743147850037),
 ('столько', 0.8770933151245117),
 ('передать', 0.8769043684005737),
 ('стесняться', 0.8745048642158508),
 ('заниматься', 0.8715790510177612),
 ('"понимать', 0.8713005781173706),
 ('нюхать', 0.8707858324050903)]

Вообще неожиданный результат. Но если так подумать, предсказать для такого абстрактного понятия что-либо тяжело. А вот если это слово представить буквально, то очень логично 'заниматься'.

Синонимов/антонимов нет.

О языке это ничего не говорит. О корпусе - он немного пошлый.

In [77]:
pos_neg_sim = model_texts.wv.most_similar(positive=["хороший"], negative=["плохой"], topn=10)
pos_neg_sim

[('очень', 0.2837032675743103),
 ('париж', 0.23415185511112213),
 ('нью', 0.21927103400230408),
 ('время', 0.2142374962568283),
 ('жить', 0.2066187560558319),
 ('некоторый', 0.18750937283039093),
 ('йорк', 0.1842970997095108),
 ('история', 0.1730724424123764),
 ('знать', 0.16012783348560333),
 ('город', 0.15698279440402985)]

Довольно логичные слова. Не оч только: нью, джуля.

Синонимы/антонимы: нет (мб нью/история, если хотите. но нью - это прилагательное вообще-то).

О языке это ничего не говорит. О корпусе - тоже ничего, разве что есть какая-то Джуля